In [1]:
import os
from pathlib import Path
from Dependencies import gvp_functions as gvp
import pandas as pd
import sqlalchemy as sql
from sqlalchemy import event
from sqlalchemy.sql import text
from shutil import move
from datetime import date
from dateutil.relativedelta import relativedelta
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep

pd.set_option('display.max_columns', None)

In [2]:
download_folder = os.path.join(Path.home(), 'Downloads')
cwd = os.getcwd()
data_folder = os.path.join(cwd, 'Data')
query_folder = os.path.join(cwd, 'Queries')

mstr_url = '' # Microstrategy URL
data_file = 'Ranking Data Reload.csv'
data_path = os.path.join(data_folder, data_file)
download_path = os.path.join(download_folder, data_file)

fiscal_ranks_file = 'Fiscal_Ranks.sql'
metric_ranks_file = 'Metric_Ranks.sql'
metric_insert_file = 'Metric_Insert.sql'
post_metric_insert_file = 'Post_metric_Insert.sql'
post_ranking_historical_file = 'Post_Ranking_Historical.sql'



In [3]:
fiscal_ranks_path = os.path.join(query_folder, fiscal_ranks_file)
metric_ranks_path = os.path.join(query_folder, metric_ranks_file)
metric_insert_path = os.path.join(query_folder, metric_insert_file)
post_metric_insert_path = os.path.join(query_folder, post_metric_insert_file)
post_ranking_historical_path = os.path.join(query_folder, post_ranking_historical_file)

In [ ]:
print('Installing Chrome Drivers')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
print('Drivers Installed')

gvp.download_reports(driver, mstr_url, data_file, export_type='csv')

move(download_path, data_path)
print(f'Data moved to {data_path}')
driver.quit()

In [4]:
data_df = pd.read_csv(data_path, encoding='UTF-16 LE')
row_count = data_df.shape[0]
print(f'Rows to upload: {row_count:,}')

Rows to upload: 136,980


In [5]:
data_df

,mgrPSID,supPSID,agentPSID,Date,Fiscal Mth,ahtNom,ahtDenom,fcrNom,fcrDenom,vocNom,vocDenom,trpDenom,TransNom,trpNom,adhDenom,adhNom,prodNom,cpcNom,auxNom,schedTime,coachAuxNom,stfdTime,trackerNom,samNom,samDenom,coachCount
0,1001266,1016508,6062472,5/3/2023,202305,"18,970",25,17,23,2,4,25,2,21,"19,800","18,568","19,695",$0.00,"2,276","19,800",0,"25,646",23,9,15,0
1,1001266,1016508,6062472,5/4/2023,202305,"18,217",30,22,26,0,0,30,4,28,"19,500","18,719","18,208",$0.00,"6,164","19,500",0,"28,764",29,13,14,0
2,1001266,1016508,6062472,5/5/2023,202305,0,0,0,0,0,0,0,0,0,0,0,0,$0.00,0,0,0,0,0,0,0,0
3,1001266,1016508,6062472,5/6/2023,202305,"22,781",31,19,23,4,4,31,8,26,"26,700","24,592","25,806",$0.00,"2,318","26,700",0,"32,125",27,5,12,0
4,1001266,1016508,6062472,5/7/2023,202305,0,0,0,0,0,0,0,0,0,0,0,0,$0.00,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136975,6181286,6133253,6090088,5/22/2023,202305,0,0,0,0,0,0,0,0,0,0,0,0,$0.00,"43,496",0,0,"45,342",0,0,0,0
136976,6181286,6133253,6090088,5/23/2023,202305,0,0,0,0,0,0,0,0,0,"26,700",0,0,$0.00,"78,801","26,700",0,"80,617",0,0,0,0
136977,6181286,6133253,6090088,5/24/2023,202305,0,0,0,0,0,0,0,0,0,0,0,0,$0.00,"35,821",0,0,"35,821",0,0,0,0
136978,6263969,6263969,6263969,5/30/2023,202306,0,0,0,0,0,0,0,0,0,0,0,0,$0.00,0,0,0,0,0,0,0,0


In [6]:
data_df = data_df.drop(columns='coachCount')

In [7]:
def clean_strings(string):
    clean_string = string
    if ',' in clean_string:
        clean_string = clean_string.replace(',', '')
    
    if (clean_string[0] == '(') and (clean_string[-1] == ')'):
        clean_string = clean_string[1:-1]
    if '$' in clean_string:
        clean_string = clean_string.replace('$', '')
    return clean_string

In [9]:
int_columns = [value for value in data_df.columns if value.endswith('PSID') or (value == 'Fiscal Mth')]
float_columns = [value for value in data_df.columns if (value != 'Date') and (value not in int_columns)]


for column in int_columns:
    # print(column)
    data_df[column] = data_df[column].astype(int)
for column in float_columns:
    # print(column)
    data_df[column] = data_df[column].map(lambda x: clean_strings(x) if type(x) == str else x)
    data_df[column] = data_df[column].astype(float)

data_df['Date'] = pd.to_datetime(data_df['Date']).dt.date

In [ ]:
data_df

In [11]:
data_df.dtypes

mgrPSID          int32
supPSID          int32
agentPSID        int32
Date            object
Fiscal Mth       int32
ahtNom         float64
ahtDenom       float64
fcrNom         float64
fcrDenom       float64
vocNom         float64
vocDenom       float64
trpDenom       float64
TransNom       float64
trpNom         float64
adhDenom       float64
adhNom         float64
prodNom        float64
cpcNom         float64
auxNom         float64
schedTime      float64
coachAuxNom    float64
stfdTime       float64
trackerNom     float64
samNom         float64
samDenom       float64
dtype: object

In [12]:
server_name = '' # Network Server Address
db_name = 'GVPOperations'

engine = sql.create_engine(
    f'mssql+pyodbc://@{server_name}/{db_name}?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server', fast_executemany=True)
conn = engine.connect()
print('Database connected')

# updating the receive_before_cursor method to use fast_executemany in order to insert faster rather than one row at a time
@event.listens_for(engine, "before_cursor_execute")
def receive_before_cursor_execute(
    conn, cursor, statement, params, context, executemany
):
    if executemany:
        cursor.fast_executemany = True

Database connected


In [13]:
clean_table_query = f'''
DELETE FROM GVPOperations.VID.R_Data;
'''

conn.execute(text(clean_table_query))
conn.commit()

In [14]:
# writing data to table from dataframe before committing the changes and closing the connection to server.
print(f'Writing {row_count:,} rows to database. (This can take a few minutes)')
data_df.to_sql(name='R_Data', con=engine,
          schema='VID', if_exists='append', index=False)
conn.close()
engine.dispose()
print('Writing completed.')

Writing 136,980 rows to database. (This can take a few minutes)
Writing completed.


In [ ]:
yesterday = date.today() - relativedelta(days=1)
fiscal_month = gvp.decide_fm(yesterday)
fiscal_str = fiscal_month.strftime("%Y%m")

In [ ]:
print(fiscal_str)

In [ ]:
import pyodbc
conn_str = ("Driver={SQL Server};"
            "Server=;" # Network Server Address
            #"Database=Aspect;"
            "Trusted_Connection=yes;")

# creating connection to server
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
print('Connecting to Server')

In [ ]:
# reading the query in as a string
with open(fiscal_ranks_path, 'r') as file:
    query_fiscal_ranks = file.read()
with open(metric_ranks_path, 'r') as file:
    query_metric_ranks = file.read() 
with open(metric_insert_path, 'r') as file:
    query_metric_insert = file.read()
with open(post_metric_insert_path, 'r') as file:
    query_post_metric_insert = file.read()
with open(post_ranking_historical_path, 'r') as file:
    query_post_ranking_historical = file.read()



In [ ]:
no_count = '''SET NOCOUNT ON;'''

In [ ]:
query_list = [query_metric_insert, 
              query_metric_ranks, 
              query_fiscal_ranks,
              query_post_metric_insert,
              query_metric_ranks, 
              query_post_ranking_historical]

for query in query_list:
    print('-'*25)
    updated_query = query.replace('<<>>', fiscal_str)
    print('Updated Fiscal Month')
    print('Executing Query')
    cmd = (no_count + updated_query)
    cursor.execute(cmd)
    count = 1
    while cursor.nextset():
        if count >= 360:
            print('Timed Out')
            break
        print('Sleeping')
        sleep(1)

    cursor.commit()
    # conn.execute(text(updated_query))
    # conn.commit()
    print('Query Executed')
else:
    print('Update Complete. Connection Severed.')
